In [17]:
import sys

import pandas as pd
import numpy as np
#import matplotlib as plt
#from matplotlib import pyplot
#import scipy as sp
#from xgboost import XGBClassifier
import sklearn
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import mean_squared_error, r2_score

from sklearn import svm

from sklearn import preprocessing

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


import joblib

In [2]:
filename = 'events.csv'
events = pd.read_csv('./data/' + filename)

In [3]:
goals = events[(events.is_goal == 1)]
first_goals = goals.groupby("id_odsp").first();
first_goals

,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,player,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
id_odsp,,,,,,,,,,,,,,,,,,,,,
004f4ING/,004f4ING64,64,82,"Goal! Goal! Southampton 0, Swansea Cit...",1,12.0,2,Swansea,Southampton,jonjo shelvey,...,NaN,NaN,13.0,1.0,1,15.0,1.0,1,1.0,0
00LMl81F/,00LMl81F20,20,17,"Goal! Roma 0, Milan 1. Zlatan Ibrahimovic (Mi...",1,12.0,2,AC Milan,AS Roma,zlatan ibrahimovic,...,NaN,NaN,4.0,1.0,1,3.0,3.0,2,1.0,0
00QH2XdM/,00QH2XdM20,20,14,"Goal! Reims 1, Evian Thonon Gaillard 0. Diego...",1,12.0,1,Stade de Reims,Evian Thonon Gaillard,diego,...,NaN,NaN,4.0,1.0,1,3.0,2.0,0,1.0,0
00QL4t1L/,00QL4t1L87,87,77,"Goal! Borussia Dortmund 1, FC Ingolstadt 04 0...",1,12.0,1,Borussia Dortmund,FC Ingolstadt 04,pierreemerick aubameyang,...,NaN,NaN,4.0,1.0,1,13.0,3.0,2,1.0,0
00WAhrVe/,00WAhrVe25,25,28,"Goal! Borussia Monchengladbach 0, Hamburger S...",1,12.0,2,Hamburg SV,Borussia Monchengladbach,jacques zoua daogari,...,NaN,NaN,3.0,1.0,1,13.0,3.0,2,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zyEQMq2D/,zyEQMq2D13,13,18,"Goal! Real Sociedad 0, FC Sevilla 1. Jairo (F...",1,12.0,2,Sevilla,Real Sociedad,jairo,...,NaN,NaN,3.0,1.0,1,3.0,1.0,4,1.0,0
zyJsFDYG/,zyJsFDYG44,44,44,"Goal! Hannover 1, Wolfsburg 0. Mame Biram Dio...",1,12.0,1,Hannover 96,VfL Wolfsburg,mame biram diouf,...,NaN,NaN,3.0,1.0,1,3.0,2.0,1,1.0,1
zyKwAQxf/,zyKwAQxf16,16,31,"Goal! Celta de Vigo 1, MA¡laga 0. Nolito (Cel...",1,12.0,1,Celta Vigo,Malaga,nolito,...,NaN,NaN,4.0,1.0,1,13.0,1.0,1,1.0,1


In [4]:
moments = first_goals.iloc[:,[2,6,7,8]].copy();

col = 'time'
conditions = [moments[col] < 25 , moments[col] <=45 , moments[col] <=60 , moments[col] > 60]
choices = [1 , 2 , 3, 4]
moments['time_moment'] = np.select(conditions,choices,default=np.nan)
moments

,time,side,event_team,opponent,time_moment
id_odsp,,,,,
004f4ING/,82,2,Swansea,Southampton,4.0
00LMl81F/,17,2,AC Milan,AS Roma,1.0
00QH2XdM/,14,1,Stade de Reims,Evian Thonon Gaillard,1.0
00QL4t1L/,77,1,Borussia Dortmund,FC Ingolstadt 04,4.0
00WAhrVe/,28,2,Hamburg SV,Borussia Monchengladbach,2.0
...,...,...,...,...,...
zyEQMq2D/,18,2,Sevilla,Real Sociedad,1.0
zyJsFDYG/,44,1,Hannover 96,VfL Wolfsburg,2.0
zyKwAQxf/,31,1,Celta Vigo,Malaga,2.0


In [5]:

data = pd.DataFrame();
data['time_moment'] = 0
for i, j in moments.iterrows(): 
    if j['event_team'] not in data.columns:
            data[j['event_team']] = 0
    if j['opponent'] not in data.columns:
        data[j['opponent']] = 0
    if j['side'] == 1:
        data = data.append({'time_moment': j['time_moment'], j['event_team'] : 1 , j['opponent']: -1 }, ignore_index=True)
    else:
        data = data.append({'time_moment': j['time_moment'], j['opponent'] : 1 , j['event_team']: -1 }, ignore_index=True)

data.head()




,time_moment,Swansea,Southampton,AC Milan,AS Roma,Stade de Reims,Evian Thonon Gaillard,Borussia Dortmund,FC Ingolstadt 04,Hamburg SV,...,US Pescara,Mallorca,Carpi,QPR,Crotone,Lecce,Cardiff,Middlesbrough,Leganes,Livorno
0,4.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,NaN,NaN,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,NaN,NaN,NaN,NaN,1.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
data = data.replace(np.nan, 0)
data

,time_moment,Swansea,Southampton,AC Milan,AS Roma,Stade de Reims,Evian Thonon Gaillard,Borussia Dortmund,FC Ingolstadt 04,Hamburg SV,...,US Pescara,Mallorca,Carpi,QPR,Crotone,Lecce,Cardiff,Middlesbrough,Leganes,Livorno
0,4.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8355,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8356,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8357,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8358,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
X = data.iloc[:,1:]
y = data.iloc[:,:1]
#y = np.ravel(y)
print(X.shape)
print(y.shape)
y

(8360, 142)
(8360, 1)


,time_moment
0,4.0
1,1.0
2,1.0
3,4.0
4,2.0
...,...
8355,1.0
8356,2.0
8357,2.0
8358,1.0


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=1)

In [20]:
#clf = RandomForestClassifier(n_estimators=500)
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)

c:\users\gustavo\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [21]:
y_pred = clf.predict(X_test)

print(r2_score(y_test, y_pred))

print(mean_squared_error(y_test, y_pred))


-0.7242316903303887
1.9477101845522897


In [22]:
accuracy = clf.score(X_test, y_test)
#y_pred = clf.predict_proba(X_test)
#predict = clf.predict(X_test)
#y_total = y_train.count()
#y_positive = y_train.sum()
#print('The training set contains {} examples (games) of which {} are positives .'.format(y_total, y_positive))
print('The accuracy of classifying is {:.2f} %'.format(accuracy*100))

The accuracy of classifying is 47.85 %


In [13]:
mlp = clf
mlp.score(X_train, y_train)
mlp.score(X_test, y_test)
accuracy = mlp.score(X_test, y_test)
print('The accuracy of classifying whether a shot is goal or not is {:.2f} %.'.format(accuracy*100))
y_pred = mlp.predict_proba(X_test)
predict = mlp.predict(X_test)
y_total = y_train.count()
y_positive = y_train.sum()
print('The training set contains {} examples of which {} are positives.'.format(y_total, y_positive))
auc_roc = roc_auc_score(y_test, y_pred[:,1])
print('Our MLP classifier obtains an AUC-ROC of {:.4f}.'.format(auc_roc))
auc_pr_baseline = y_positive / y_total
print('The baseline performance for AUC-PR is {:.4f}. This is what we would get by random guessing'.format(auc_pr_baseline))
auc_pr = average_precision_score(y_test, y_pred[:,1])
print('Our MLP classifier obtains an AUC-PR of {:.4f}.'.format(auc_pr))
cohen_kappa = cohen_kappa_score(y_test,predict)
print('Our classifier obtains a Cohen Kappa of {:.4f}.'.format(cohen_kappa))
MSE = sklearn.metrics.mean_squared_error(y_test, y_pred[:,1])
print('Our MLP classifier obtains a Mean Squared Error (MSE) of {:.4f}.'.format(MSE))

The accuracy of classifying whether a shot is goal or not is 36.71 %.
The training set contains time_moment    5434
dtype: int64 examples of which time_moment    10403.0
dtype: float64 are positives.


ValueError: multi_class must be in ('ovo', 'ovr')

In [ ]:
print('Confusion Matrix:')
print(confusion_matrix(y_test,predict))
print('Report:')
print(classification_report(y_test,predict))
